In [14]:
!pip install wandb -qU

In [ ]:
import os
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications.resnet import ResNet50
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import wandb
from wandb.keras import WandbCallback
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform

In [23]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [24]:
# Initialize wandb
wandb.init(project='ci-part2', entity='radical-p')

accuracy,▁▅▇██▁▅▇██▂▅▆█
epoch,▁▃▅▆█▁▃▅▆█▁▃▅▆
loss,█▄▂▂▁█▄▂▁▁█▄▃▁
val_accuracy,▄▁▅▅▇██▆▂▄▇▂▅▅
val_loss,▄█▄▄▂▁▁▃▆▄▂▆▄▄
accuracy,0.66889
best_epoch,1
best_val_loss,0.33271
epoch,3
loss,0.84391
val_accuracy,0.65539


In [ ]:
# Define the path to the train, test, and validation folders
train_path = '/content/drive/MyDrive/output_folder/HW02-Practical/Dataset/train'
test_path = '/content/drive/MyDrive/output_folder/HW02-Practical/Dataset/test'
val_path = '/content/drive/MyDrive/output_folder/HW02-Practical/Dataset/valid'

In [28]:
import cv2
# Define the image size
#img_size = (112, 112)
img_size = (56, 56)

# Define the number of classes
num_classes = 5

# Define the class names and their corresponding integer labels
class_names = ['articulated_truck', 'background', 'bus', 'car', 'work_van']
class_dict = {class_name: i for i, class_name in enumerate(class_names)}

# Load the images and labels from the train folder
train_images = []
train_labels = []
for folder in os.listdir(train_path):
    folder_path = os.path.join(train_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, img_size)
        img = img.astype('float32') / 255.0
        train_images.append(img)
        train_labels.append(class_dict[folder])

# Load the images and labels from the test folder
test_images = []
test_labels = []
for folder in os.listdir(test_path):
    folder_path = os.path.join(test_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, img_size)
        img = img.astype('float32') / 255.0
        test_images.append(img)
        test_labels.append(class_dict[folder])

# Load the images and labels from the validation folder
val_images = []
val_labels = []
for folder in os.listdir(val_path):
    folder_path = os.path.join(val_path, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, img_size)
        img = img.astype('float32') / 255.0
        val_images.append(img)
        val_labels.append(class_dict[folder])


# Convert the images and labels to arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)

# One-hot encode the labels
enc = OneHotEncoder(categories='auto')
train_labels = enc.fit_transform(train_labels.reshape(-1, 1)).toarray()
test_labels = enc.transform(test_labels.reshape(-1, 1)).toarray()
val_labels = enc.transform(val_labels.reshape(-1, 1)).toarray()

In [ ]:
# Define the ResNet18 model
from keras.wrappers.scikit_learn import KerasClassifier

def create_model(lr=1e-3, num_classes=5):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3))
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model with the given hyperparameters
    optimizer = Adam(lr=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Wrap the Keras model in a scikit-learn KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
# Define the hyperparameters to tune
'''
param_dist = {
    'lr': loguniform(1e-4, 1e-2),
    'batch_size': [8, 32, 64],
    'epochs': [5, 10, 20]
}
'''
param_dist = {
    'lr': loguniform(1e-4, 1e-2),
    'batch_size': [32],
    'epochs': [5]
}

In [ ]:
# Run theparameter tuning with random search
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=3, cv=3, n_jobs=-1, verbose=1)
random_search.fit(train_images, train_labels)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


In [19]:
# Evaluate the model on the test set with the best hyperparameters
lr = random_search.best_params_['lr']
batch_size = random_search.best_params_['batch_size']
epochs = random_search.best_params_['epochs']
model = create_model(lr=lr, num_classes=num_classes)
model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_labels), callbacks=[WandbCallback(), EarlyStopping(patience=3)])
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/5
469/469 [==============================] - 209s 436ms/step - loss: 1.1137 - accuracy: 0.5506 - val_loss: 0.6010 - val_accuracy: 0.8182
Epoch 2/5
469/469 [==============================] - 198s 422ms/step - loss: 0.9502 - accuracy: 0.6244 - val_loss: 1.3918 - val_accuracy: 0.3975
Epoch 3/5
469/469 [==============================] - 182s 388ms/step - loss: 0.9047 - accuracy: 0.6452 - val_loss: 0.9068 - val_accuracy: 0.6998
Epoch 4/5
157/157 [==============================] - 58s 370ms/step - loss: 0.8937 - accuracy: 0.6622
Test accuracy: 0.6621999740600586


In [21]:
def create_model_learnable(lr=1e-3, num_classes=5):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3))
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = True

    # Compile the model with the given hyperparameters
    optimizer = Adam(lr=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

<ipython-input-21-31b19f1ff459>:19: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_learnable = KerasClassifier(build_fn=create_model, verbose=0)


In [26]:
model_learnable = create_model_learnable(lr=lr, num_classes=num_classes)
model_learnable.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_labels), callbacks=[WandbCallback(), EarlyStopping(patience=3)])
test_loss_learnable, test_acc_learnable = model_learnable.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc_learnable)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
469/469 [==============================] - 2042s 4s/step - loss: 2.5122 - accuracy: 0.3502 - val_loss: 6.6333 - val_accuracy: 0.0085
Epoch 2/5
469/469 [==============================] - 2057s 4s/step - loss: 1.1802 - accuracy: 0.5804 - val_loss: 1.1080 - val_accuracy: 0.5307
Epoch 3/5
469/469 [==============================] - ETA: 0s - loss: 0.8177 - accuracy: 0.7020

wandb: Adding directory to artifact (/content/wandb/run-20230517_115709-pw39nktr/files/model-best)... Done. 1.7s


469/469 [==============================] - 2442s 5s/step - loss: 0.8177 - accuracy: 0.7020 - val_loss: 0.8841 - val_accuracy: 0.6617
Epoch 4/5
469/469 [==============================] - 2332s 5s/step - loss: 0.6157 - accuracy: 0.7771 - val_loss: 1.2261 - val_accuracy: 0.5899
Epoch 5/5
469/469 [==============================] - ETA: 0s - loss: 0.5125 - accuracy: 0.8205

wandb: Adding directory to artifact (/content/wandb/run-20230517_115709-pw39nktr/files/model-best)... Done. 3.5s


157/157 [==============================] - 70s 448ms/step - loss: 0.8937 - accuracy: 0.6622
Test accuracy: 0.6621999740600586


In [27]:
test_loss_learnable, test_acc_learnable = model_learnable.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc_learnable)

157/157 [==============================] - 79s 501ms/step - loss: 0.8100 - accuracy: 0.7270
Test accuracy: 0.7269999980926514
